In [1]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
from data import get_dichalcogenides_innopolis_202105

In [248]:
from pymatgen.core.sites import PeriodicSite
from pymatgen.core.periodic_table import DummySpecies

In [2]:
structures = get_dichalcogenides_innopolis_202105()

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [304]:
# TODO(inner perfectionist) eval is unsecure
defects = pd.read_csv(
  "datasets/dichalcogenides_innopolis_202105/descriptors.csv", index_col="_id",
  converters={"cell": eval, "defects": eval})

In [305]:
materials = defects.base.unique()

In [306]:
unit_cells = {}
for material in materials:
  unit_cells[material] = pymatgen.io.cif.CifParser(os.path.join(
  "defects_generation/molecules", f"{material}.cif")).get_structures(primitive=True)[0]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [307]:
def get_frac_coords_set(structure):
  return set(map(tuple, np.round(structure.frac_coords, 3)))

In [308]:
def get_defects(structure, unit_cell, supercell):
  reference_species = set(unit_cell.species)
  reference_supercell = unit_cell.copy()
  reference_supercell.make_supercell(supercell)
  reference_sites = get_frac_coords_set(reference_supercell)

  defects = []
  for site in structure:
    if site.specie not in reference_species:
      defects.append(site.specie)
  absent_sites = reference_sites - get_frac_coords_set(structure)
  defects.extend(map(lambda coords: PeriodicSite(
    species=DummySpecies(), coords=coords, coords_are_cartesian=False, lattice=structure.lattice), absent_sites))
  return defects

In [309]:
def get_defecs_from_row(row):
  this_defect = defects.loc[row.descriptor_id]
  return get_defects(row.initial_structure, unit_cells[this_defect.base], this_defect.cell)

In [310]:
structures["defect_representation"] = structures.apply(get_defecs_from_row, axis=1)

In [312]:
# Test
assert structures.apply(
  lambda row: len(row.defect_representation) == len(defects.loc[row.descriptor_id, "defects"]), 
  axis=1).all()